In [1]:
# -*- coding: utf-8 -*-
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.linear_model import Lasso    #Regressão Linear com L1
from sklearn.linear_model import LinearRegression  #Regressão Linear Simples
from sklearn.model_selection import train_test_split


#Função que recebe os Ys calculados pela regressão, que são os Percents, e os Ys reais que são os PL (perdas/lucros) de cada aposta
def SomaLog(Percent,PL):
    MIN_BANCA,MAX_BANCA= 0.01, 0.1
    return sum((MAX_BANCA if p>MAX_BANCA else p)*pl for p,pl in zip(Percent,PL) if p>=MIN_BANCA )



#Le o csv e filtra as colunas que serão utilizadas
df=pd.read_csv('../input/jogos_.csv')
df=df['s_g,s_c,s_da,s_s,s_r,d_g,d_c,d_da,d_s,goal,goal_diff,oddsU,probU,probU_diff,mod0,mod25,mod50,mod75,pl_u'.split(',')]

#Remove as linhas estão NaN, (preguiça de procurar, mas como é só um exemplo de uso tudo bem)
df.dropna(inplace=True)

#Mostra as 5 primeiras linhas
df.head(5)

,s_g,s_c,s_da,s_s,s_r,d_g,d_c,d_da,d_s,goal,goal_diff,oddsU,probU,probU_diff,mod0,mod25,mod50,mod75,pl_u
0,0.0,6.0,54.0,8.0,0.0,0.0,4.0,12.0,6.0,1.25,1.25,1.925,0.493421,0.006579,0.0,1.0,0.0,0.0,0.4625
1,0.0,2.0,66.0,6.0,0.0,0.0,2.0,10.0,4.0,1.25,1.25,1.950,0.486842,0.013158,0.0,1.0,0.0,0.0,-1.0000
2,1.0,8.0,34.0,9.0,0.0,1.0,8.0,0.0,3.0,2.50,1.50,1.750,0.539474,0.039474,0.0,0.0,1.0,0.0,-1.0000
3,3.0,9.0,52.0,6.0,0.0,1.0,5.0,2.0,0.0,4.25,1.25,1.975,0.480263,0.019737,0.0,1.0,0.0,0.0,-1.0000
4,2.0,4.0,28.0,11.0,0.0,0.0,0.0,6.0,5.0,3.50,1.50,1.900,0.500000,0.000000,0.0,0.0,1.0,0.0,-1.0000


Vamos dividir df entre conjunto de Treinamento e Teste de forma randômica, mas fixando o tamanho entre 50 mil registros. Será escolhido um estado randômico de forma que a média do pl_u do conjunto de teste esteja bem próxima da média do conjunto de treinamento.

In [2]:
#Divide entre treinamento e teste, foi escolhi o random_state=881, pois nesse as médias de Y_train e  Y_test estão bem próximas
X_train, X_test, Y_train, Y_test = train_test_split(df.loc[:,df.columns!='pl_u'], df.pl_u, test_size=50000, random_state=881)

print('pl_medio_train:', Y_train.mean(), '\npl_medio_test: ', Y_test.mean())

pl_medio_train: -0.02396081235105205 
pl_medio_test:  -0.023993450000000093


Comos visto no noteboook anterior https://www.kaggle.com/ronaldoaf/lasso-regression-com-lucratividade quanto menor o alpha mais lucrativa a regressão. Então vvamos fazer uma Lasso Regression bem pequeno, alpha=1e-5 e ver como ficam os coeficientes da equação e a lucratividade 

In [3]:
#Treinamos a regressão com alpha=1e-5  ( ou seja, 0.00001)
reg1=Lasso(alpha=1e-5, max_iter=10000).fit(X_train,Y_train)

#Exibe a equação
print(  'pl_u= '+' + '.join("{0:.6}".format(reg1.coef_[i])+'*'+col for i,col in enumerate(X_train.columns)) +" + {0:.6}".format(reg1.intercept_)  )

#Exibe a Lucratividade através do SomaLog
print( '\nSomaLog: ', SomaLog( reg1.predict(X_test), Y_test)  )

pl_u= -0.00383955*s_g + -0.00475556*s_c + -0.00027072*s_da + -0.00569096*s_s + -0.00625057*s_r + -0.0266456*d_g + -4.11758e-05*d_c + -0.000623279*d_da + -0.00126863*d_s + 0.0*goal + 0.0846958*goal_diff + 0.100175*oddsU + 0.147322*probU + -0.155697*probU_diff + 0.01233*mod0 + 0.004332*mod25 + -3.71935e-06*mod50 + -0.0143503*mod75 + -0.282762

SomaLog:  25.251176179527583


Agora vamos fazer uma Regressão Linear Simples


In [4]:
#Treinamos a regressão com alpha=1e-4  ( ou seja, 0.0001)
reg2=LinearRegression().fit(X_train,Y_train)

#Exibe a equação
print(  'pl_u= '+' + '.join("{0:.6}".format(reg2.coef_[i])+'*'+col for i,col in enumerate(X_train.columns)) +" + {0:.6}".format(reg2.intercept_)  )

#Exibe a Lucratividade através do SomaLog
print( '\nSomaLog: ', SomaLog( reg2.predict(X_test), Y_test)  )

pl_u= 8.64646e+09*s_g + -0.00479931*s_c + -0.000264793*s_da + -0.00574201*s_s + -0.00654133*s_r + -0.0266502*d_g + -4.71275e-06*d_c + -0.000647276*d_da + -0.00126093*d_s + -8.64646e+09*goal + 8.64646e+09*goal_diff + 0.330795*oddsU + 1.05631*probU + -0.270696*probU_diff + -6.09903e+07*mod0 + -6.09903e+07*mod25 + -6.09903e+07*mod50 + -6.09903e+07*mod75 + 6.09903e+07

SomaLog:  25.3739214558526


Conclusão que com um conjunto de treinamento bem grande a Regressão Linear Simples é melhor que a Lasso. E muito mais rápida